# Hypothesis Testing


We would like to know if the effects we see in the sample(observed data) are likely to occur in the population. 

The way classical hypothesis testing works is by conducting a statistical test to answer the following question:
> Given the sample and an effect, what is the probability of seeing that effect just by chance?

Here are the steps on how we would do this

1. Compute test statistic
2. Define null hypothesis
3. Compute p-value
4. Interpret the result

If p-value is very low(most often than now, below 0.05), the effect is considered statistically significant. That means that effect is unlikely to have occured by chance. The inference? The effect is likely to be seen in the population too. 

This process is very similar to the *proof by contradiction* paradigm. We first assume that the effect is false. That's the null hypothesis. Next step is to compute the probability of obtaining that effect (the p-value). If p-value is very low(<0.05 as a rule of thumb), we reject the null hypothesis. 

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib as mpl
%matplotlib inline

In [2]:
import seaborn as sns
sns.set(color_codes=True)

In [3]:
weed_pd = pd.read_csv("../data/Weed_Price.csv", parse_dates=[-1])

In [4]:
weed_pd["month"] = weed_pd.date.apply(lambda x: x.month)
weed_pd["year"] = weed_pd.date.apply(lambda x: x.year)

In [5]:
weed_pd.head()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date,month,year
0,Alabama,339.06,1042,198.64,933,149.49,123,2014-01-01,1,2014
1,Alaska,288.75,252,260.60,297,388.58,26,2014-01-01,1,2014
2,Arizona,303.31,1941,209.35,1625,189.45,222,2014-01-01,1,2014
3,Arkansas,361.85,576,185.62,544,125.87,112,2014-01-01,1,2014
4,California,248.78,12096,193.56,12812,192.92,778,2014-01-01,1,2014


### Let's work on weed prices in California in 2014


In [6]:
weed_ca_2014 = weed_pd[(weed_pd.State=="California") & (weed_pd.year==2014)]

In [7]:
#Mean and standard deviation of high quality weed's price
print "Mean:", weed_ca_2014.HighQ.mean()
print "Standard Deviation:", weed_ca_2014.HighQ.std()

Mean: 245.894230769
Standard Deviation: 1.28990793937


In [8]:
#Confidence interval on the mean
stats.norm.interval(0.95, loc=weed_ca_2014.HighQ.mean(), scale = weed_ca_2014.HighQ.std()/np.sqrt(len(weed_ca_2014)))

(245.761718492726, 246.02674304573577)

### Question: Are high-quality weed prices in Jan 2014 significantly higher than in Jan 2015?

In [9]:
#Get the data
weed_ca_jan2014 = np.array(weed_pd[(weed_pd.State=="California") & (weed_pd.year==2014) & (weed_pd.month==1)].HighQ)
weed_ca_jan2015 = np.array(weed_pd[(weed_pd.State=="California") & (weed_pd.year==2015) & (weed_pd.month==1)].HighQ)

In [10]:
print "Mean-2014 Jan:", weed_ca_jan2014.mean()
print "Mean-2015 Jan:", weed_ca_jan2015.mean()

Mean-2014 Jan: 248.445483871
Mean-2015 Jan: 243.602258065


In [11]:
print "Effect size:", weed_ca_jan2014.mean() - weed_ca_jan2015.mean()

Effect size: 4.84322580645


**Null Hypothesis**: Mean prices aren't significantly different

Perform **t-test** and determine the p-value. 

In [12]:
stats.ttest_ind(weed_ca_jan2014, weed_ca_jan2015, equal_var=True)

Ttest_indResult(statistic=98.011325238158051, pvalue=6.2979718185084028e-68)

p-value is the probability that the effective size was by chance. And here, p-value is almost 0.

*Conclusion*: The price difference is significant. But is a price increase of $4.85 a big deal? The price decreased in 2015 by almost 2%. Always remember to look at effect size. 

**Problem** Determine if prices of medium quality weed for Jan 2015 and Feb 2015 are significantly different for New York. 

### Assumption of t-test

One assumption is that the data used came from a normal distribution. 
<br>
There's a [Shapiro-Wilk test](https://en.wikipedia.org/wiki/Shapiro-Wilk) to test for normality. If p-value is less than 0.05, then there's a low chance that the distribution is normal.

In [13]:
stats.shapiro(weed_ca_jan2015)

(0.9469053149223328, 0.12818680703639984)

In [14]:
stats.shapiro(weed_ca_jan2014)

(0.9353488683700562, 0.06141229346394539)

In [ ]:
#We seem to be good.

### A/B testing

Comparing two versions to check which one performs better. Eg: Show to people two variants for the same webpage that they want to see and find which one provides better conversion rate (or the relevant metric). [wiki](https://en.wikipedia.org/wiki/A/B_testing)

**Exercise: Impact of regulation and deregulation.**

Information on regulation of Weed in the US by State [wiki](Impact of regulation and deregulation on a couple of states )

1. Alaska legalized it on 4th Nov 2014. Find if prices significantly changed in Dec 2014 compared to Oct 2014. 
2. Maryland decriminalized possessing weed from Oct 1, 2014. Find if prices of weed changed significantly in Oct 2014 compared to Sep 2014

<h2> Something to think about: Which of these give smaller p-values ? </h2>
   
   * Smaller effect size
   * Smaller standard error
   * Smaller sample size
   * Higher variance
   
   **Answer:** 

### Chi-square tests

Chi-Square tests are used when the data are frequencies, rather than numerical score/price.

The following two tests make use of chi-square statistic

1. chi-square test for goodness of fit
2. chi-square test for independence

Chi-square test is a non-parametric test. They do not require assumptions about population parameters and they do not test hypotheses about population parameters.

<h2> Chi-Square test for goodness fit </h2>

$$ \chi^2 = \sum (O - E)^2/E $$

* O is observed frequency
* E is expected frequency
* $ \chi $ is the chi-square statistic

Let's assume the proportion of people who bought High, Medium and Low quality weed in Jan-2014 as the expected proportion. Find if proportion of people who bought weed in Jan 2015 conformed to the norm

In [16]:
weed_jan2014 = weed_pd[(weed_pd.year==2014) & (weed_pd.month==1)][["HighQN", "MedQN", "LowQN"]]
weed_jan2015 = weed_pd[(weed_pd.year==2015) & (weed_pd.month==1)][["HighQN", "MedQN", "LowQN"]]

In [17]:
Expected = np.array(weed_jan2014.apply(sum, axis=0))
Observed = np.array(weed_jan2015.apply(sum, axis=0))

In [18]:
print "Expected:", Expected, "\n" , "Observed:", Observed

Expected: [2918004 2644757  263958] 
Observed: [4057716 4035049  358088]


In [19]:
print "Expected:", Expected/np.sum(Expected.astype(float)), "\n" , "Observed:", Observed/np.sum(Observed.astype(float))

Expected: [ 0.5007971   0.45390159  0.04530131] 
Observed: [ 0.48015461  0.47747239  0.042373  ]


In [20]:
stats.chisquare(Observed, Expected)

Power_divergenceResult(statistic=1209562.2775169075, pvalue=0.0)

*Inference* : We reject null hypothesis. The proportions in Jan 2015 is different than what was expected.

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Statistics using Python</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://twitter.com/bargava/" property="cc:attributionName" rel="cc:attributionURL">Bargava</a> and <a xmlns:cc="http://creativecommons.org/ns#" href="https://twitter.com/raghothams/" property="cc:attributionName" rel="cc:attributionURL">Raghotham</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.